# Time Series Analysis Part 2
Fitting the model from Part 1 for all available suburbs, and forecasting their growth over next 3 years.
Written by Daksh Agrawal.

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from pandas.tseries.offsets import DateOffset
import warnings

In [46]:
warnings.filterwarnings('ignore')

In [47]:
FORECAST_PERIOD = 12  # 12 Quarters

In [48]:
# Load the data
df = pd.read_csv('../data/curated/historical_data.csv')
df

,type,suburb,Count,Date,Median,year_completed,sa2_name,sa2_code,Unnamed: 0,SA2_CODE,Population,percentage_change_rental_price,percentage_change_population
0,1 bedroom flat,Altona,87.0,2000-03-01,95.0,2000,Altona,213021341,930,213021341,12297.095890,NaN,NaN
1,1 bedroom flat,Altona,94.0,2000-06-01,100.0,2000,Altona,213021341,1452,213021341,12271.915068,0.052632,-0.002048
2,1 bedroom flat,Altona,97.0,2000-09-01,105.0,2000,Altona,213021341,1974,213021341,12246.457534,0.050000,-0.002074
3,1 bedroom flat,Altona,98.0,2000-12-01,105.0,2000,Altona,213021341,2496,213021341,12221.000000,0.000000,-0.002079
4,1 bedroom flat,Altona,89.0,2001-03-01,105.0,2001,Altona,213021341,3018,213021341,12196.095890,0.000000,-0.002038
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29619,All properties,Wodonga,1233.0,2022-03-01,380.0,2022,Wodonga,204031492,46538,204031492,14863.726027,0.027027,-0.001094
29620,All properties,Wodonga,1267.0,2022-06-01,390.0,2022,Wodonga,204031492,47060,204031492,14847.271233,0.026316,-0.001107
29621,All properties,Wodonga,1251.0,2022-09-01,400.0,2022,Wodonga,204031492,47582,204031492,14830.635616,0.025641,-0.001120
29622,All properties,Wodonga,1191.0,2022-12-01,410.0,2022,Wodonga,204031492,48104,204031492,14814.000000,0.025000,-0.001122


In [49]:
df['Date'] = pd.to_datetime(df['Date'])

In [50]:
suburbs = df['suburb'].unique()
types = df['type'].unique()

In [51]:
full_forecast_df = pd.DataFrame()

In [52]:
def get_arima_forecast(data, order=(4, 1, 3)):
    model = ARIMA(data, order=order)
    model_fit = model.fit()
    forecast = model_fit.get_forecast(steps=FORECAST_PERIOD)
    forecast_mean = forecast.predicted_mean
    forecast_ci = forecast.conf_int()
    return forecast_mean, forecast_ci

In [53]:
num_failed = 0

for suburb in suburbs:
    for type in types:
        data = df[(df['suburb'] == suburb) & (df['type'] == type)]
        data = data[['Date', 'Median']].set_index('Date')
        data = data.dropna()

        try:
            forecast_mean, forecast_ci = get_arima_forecast(data)
        except:
            print(f'Failed to forecast {suburb} - {type}')
            num_failed += 1
            continue

        forecast_df = pd.DataFrame({
            'Date': pd.date_range(start=data.index[-1], periods=FORECAST_PERIOD, freq='QE'),
            'Median': forecast_mean.values,
            'Lower': forecast_ci['lower Median'].values,
            'Upper': forecast_ci['upper Median'].values,
            'suburb': suburb,
            'type': type
        })
        full_forecast_df = pd.concat([full_forecast_df, forecast_df])

print(f'Failed to forecast {num_failed} suburbs')

Failed to forecast Castlemaine - 3 bedroom flat
Failed to forecast Docklands - 2 bedroom house
Failed to forecast Docklands - 3 bedroom house
Failed to forecast Docklands - 4 bedroom house
Failed to forecast Keilor - 1 bedroom flat
Failed to forecast Seymour - 3 bedroom flat
Failed to forecast 6 suburbs


In [54]:
full_forecast_df

,Date,Median,Lower,Upper,suburb,type
0,2023-03-31,289.022035,280.701145,297.342924,Altona,1 bedroom flat
1,2023-06-30,288.479435,276.868189,300.090682,Altona,1 bedroom flat
2,2023-09-30,288.948965,274.518198,303.379732,Altona,1 bedroom flat
3,2023-12-31,290.866958,272.913161,308.820755,Altona,1 bedroom flat
4,2024-03-31,289.736599,268.633055,310.840143,Altona,1 bedroom flat
...,...,...,...,...,...,...
7,2024-12-31,462.832023,431.574580,494.089466,Wodonga,All properties
8,2025-03-31,468.880317,433.996520,503.764113,Wodonga,All properties
9,2025-06-30,474.307386,435.641862,512.972909,Wodonga,All properties
10,2025-09-30,481.091210,438.388929,523.793491,Wodonga,All properties


In [55]:
full_forecast_df.to_csv('../data/curated/forecast_data.csv', index=False)